In [63]:
import speech_recognition
import tempfile
from gtts import gTTS
import time
import wave
import pyaudio
from pygame import mixer
mixer.init()

def listenTo():
    
    CHUNK = 2048*20
    FORMAT = pyaudio.paInt16
    CHANNELS = 1
    RATE = 44100
    RECORD_SECONDS = 5
    WAVE_OUTPUT_FILENAME = "test.wav"

    p = pyaudio.PyAudio()

    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)

    print("* recording")

    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK, exception_on_overflow = False)
        frames.append(data)

    print("* done recording")
    stream.stop_stream()
    stream.close()
    p.terminate()

    wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    r = speech_recognition.Recognizer()
    with speech_recognition.AudioFile('test.wav') as source:
        audio = r.record(source)
    t = time.time()
    s = r.recognize_google(audio, language='cmn-Hant-TW')
    print('recognize audio', time.time()-t)
    return s
    
def speak(sentence, lang='zh-tw'):
    t = time.time()
    with tempfile.NamedTemporaryFile(delete=True) as fp:
        tts =gTTS(text=sentence, lang=lang)
        tts.save(fp.name+'.mp3')
        mixer.music.load(fp.name+'.mp3')
        mixer.music.play()
    print('text to speech', time.time()-t)
# print(listenTo())

# speak(listenTo())

In [5]:
qa = {'我想吃':'不要只會吃',
      '想睡覺':'去休息一下',
      '我愛你':'我也是'
     }
speak(qa.get(listenTo(),'人家還不能聽懂很多話'))

* recording
* done recording
recognize audio 1.4038827419281006
text to speech 1.3179740905761719


In [37]:
# 閱讀維基百科
import requests
from bs4 import BeautifulSoup

def getWiki (term):
    res = requests.get('http://zh.wikipedia.org/wiki/{}'.format(term))
    soup = BeautifulSoup(res.text, features='html.parser')
    article = soup.select_one('.mw-parser-output p').text
    
    return article

speak(getWiki(listenTo()))

* recording
* done recording
recognize audio 1.4433808326721191
text to speech 1.6072170734405518


In [66]:
from googletrans import Translator
lang = 'ja'
translator = Translator()
l = listenTo()
print(l)
s = translator.translate(l, lang, 'zh-TW').text
print(s)
speak(s, lang)

* recording
* done recording
recognize audio 1.4303851127624512
用力打電腦
コンピュータを激しく打つ
text to speech 1.2920241355895996
